# UC7 Network Connectivity

© Crown-owned copyright 2025, Defence Science and Technology Laboratory UK


This notebook is meant as supplementary material to the more expansive [UC7 E2E notebook](./UC7-E2E-Demo.ipynb).

Focusing on the simulation components of UC7, this notebook primarily intends to demonstrate the default connectivity and prove that the simulation is operating as expected. Additionally, this notebook assumes that the reader is familiar with both UC7 and PrimAITE's networking simulation as no supplementary text cells to explain specific output are provided.

## Network Description

<p align="center">
    <a href="./_package_data/uc7/uc7_network_detailed_svg.svg" target="_blank">
        <img src="./_package_data/uc7/uc7_network_detailed_svg.svg" alt="Image">
    </a>  
    
</p>


In [1]:
!primaite setup

2025-03-24 10:01:23,904: Performing the PrimAITE first-time setup...
2025-03-24 10:01:23,904: Building the PrimAITE app directories...
2025-03-24 10:01:23,904: Building primaite_config.yaml...
2025-03-24 10:01:23,904: Rebuilding the demo notebooks...
2025-03-24 10:01:23,928: Rebuilding the example notebooks...
2025-03-24 10:01:23,930: PrimAITE setup complete!


In [2]:
from primaite.session.environment import PrimaiteGymEnv
from primaite.simulator.network.hardware.nodes.host.computer import Computer
from primaite.simulator.network.hardware.nodes.host.server import Server
from primaite.simulator.network.hardware.nodes.network.router import Router
from primaite.simulator.network.hardware.nodes.network.switch import Switch
from primaite.simulator.network.container import Network
from primaite.simulator.network.hardware.nodes.network.firewall import Firewall
from primaite.game.game import PrimaiteGame
from primaite.simulator.system.services.dns.dns_client import DNSClient
from primaite.simulator.sim_container import Simulation
import yaml
from pprint import pprint
from primaite.config.load import load, _EXAMPLE_CFG

In [3]:
with open(file=_EXAMPLE_CFG/"uc7_config.yaml", mode="r") as uc7_config:
    cfg = yaml.safe_load(uc7_config)
    cfg['io_settings']['save_sys_logs'] = True # Saving syslogs
    cfg['io_settings']['save_agent_logs'] = True # Save agent logs
env = PrimaiteGymEnv(env_config=cfg)

2025-03-24 10:01:27,970: PrimaiteGymEnv RNG seed = None


In [4]:
uc7_network = env.game.simulation.network
uc7_network.show()

+------------------------------------------------------+
|                        Nodes                         |
+-------------------------+----------+-----------------+
| Node                    | Type     | Operating State |
+-------------------------+----------+-----------------+
| HOME-PUB-RT-DR          | router   | ON              |
| HOME-PUB-SW-AS          | switch   | ON              |
| HOME-PUB-PC-1           | computer | ON              |
| HOME-PUB-PC-2           | computer | ON              |
| HOME-PUB-SRV            | server   | ON              |
| ISP-PUB-RT-BR           | router   | ON              |
| ISP-PUB-SRV-DNS         | server   | ON              |
| REM-PUB-FW              | firewall | ON              |
| REM-PUB-RT-DR           | router   | ON              |
| REM-PUB-SW-AS           | switch   | ON              |
| REM-PUB-PC-1            | computer | ON              |
| REM-PUB-PC-2            | computer | ON              |
| REM-PUB-SRV             | ser

## UC7 Network | Home Office Subnet

In [5]:
home_pub_pc_1: Computer = env.game.simulation.network.get_node_by_hostname("HOME-PUB-PC-1")
home_pub_pc_2: Computer = env.game.simulation.network.get_node_by_hostname("HOME-PUB-PC-2")
home_pub_pc_srv: Server = env.game.simulation.network.get_node_by_hostname("HOME-PUB-SRV")
home_pub_rt_dr: Router = env.game.simulation.network.get_node_by_hostname("HOME-PUB-RT-DR")

In [6]:
# HOME PC 1 --> HOME PC 2
home_pub_pc_1.ping(home_pub_pc_2.network_interface[1].ip_address)

True

In [7]:
# HOME PC 2 --> HOME SERVER
home_pub_pc_2.ping(home_pub_pc_srv.network_interface[1].ip_address)

True

In [8]:
# HOME SERVER --> HOME ROUTER
home_pub_pc_srv.ping(home_pub_rt_dr.network_interface[1].ip_address)

True

## UC7 Network | Internet

In [9]:
isp_pub_rt_br: Router = env.game.simulation.network.get_node_by_hostname("ISP-PUB-RT-BR")
isp_pub_srv_dns: Server = env.game.simulation.network.get_node_by_hostname("ISP-PUB-SRV-DNS")

In [10]:
home_pub_pc_1.ping(isp_pub_srv_dns.network_interface[1].ip_address)

True

In [11]:
# HOME ROUTER --> INTERNET ROUTER
home_pub_rt_dr.ping(isp_pub_rt_br.network_interface[1].ip_address)

True

In [12]:
home_pub_rt_dr.route_table.show()

+-------------------------------------+
|      HOME-PUB-RT-DR Route Table     |
+-------+---------+----------+--------+
| Index | Address | Next Hop | Metric |
+-------+---------+----------+--------+
+-------+---------+----------+--------+


In [13]:
# HOME ROUTER --> INTERNET ISP
home_pub_rt_dr.ping(isp_pub_srv_dns.network_interface[1].ip_address)

True

In [14]:
# INTERNET ROUTER --> INTERNET DNS
isp_pub_rt_br.ping(target_ip_address=isp_pub_srv_dns.network_interface[1].ip_address)

True

In [15]:
# HOME ROUTER --> INTERNET DNS
home_pub_rt_dr.ping(target_ip_address=isp_pub_srv_dns.network_interface[1].ip_address)

True

In [16]:
# PC 1 --> INTERNET DNS

home_pub_pc_1.ping(target_ip_address=isp_pub_srv_dns.network_interface[1].ip_address)

True

## UC7 Network | Remote Site Subnet

In [17]:
network: Network = env.game.simulation.network


rem_pub_fw: Firewall = network.get_node_by_hostname(hostname="REM-PUB-FW")
rem_pub_rt_dr: Router = network.get_node_by_hostname(hostname="REM-PUB-RT-DR")
rem_pub_sw_as: Switch = network.get_node_by_hostname(hostname="REM-PUB-SW-AS")
rem_pub_pc_1: Computer = network.get_node_by_hostname(hostname="REM-PUB-PC-1")
rem_pub_pc_2: Computer = network.get_node_by_hostname(hostname="REM-PUB-PC-2")
rem_pub_srv: Computer = network.get_node_by_hostname(hostname="REM-PUB-SRV")

In [18]:
# Remote PC 1 --> Remote PC_2

rem_pub_pc_1.ping(rem_pub_pc_2.network_interface[1].ip_address)

True

In [19]:
# Remote PC 2 --> Remote Server

rem_pub_pc_2.ping(rem_pub_srv.network_interface[1].ip_address)

True

In [20]:
# Remote Server --> Remote Site Router

rem_pub_srv.ping(rem_pub_rt_dr.network_interface[1].ip_address)

True

In [21]:
# Remote Site Router --> Remote Pub Firewall (Internal)

rem_pub_rt_dr.ping(rem_pub_fw.network_interface[1].ip_address)

True

In [22]:
# Remote Site Router --> Remote Pub Firewall (External)

rem_pub_rt_dr.ping(rem_pub_fw.network_interface[2].ip_address)

True

In [23]:
# Remote Pub Firewall (external) --> Public Internet

rem_pub_fw.ping(isp_pub_rt_br.network_interface[1].ip_address)

True

In [24]:
# Remote Site PC 1 --> Public DNS

rem_pub_pc_1.ping(target_ip_address=isp_pub_srv_dns.network_interface[1].ip_address)

True

In [25]:
# Remote Site PC 1 --> Home Office PC 1

rem_pub_pc_1.ping(target_ip_address=home_pub_pc_1.network_interface[1].ip_address)

True

## UC7 Network | SOME_TECH_DMZ

In [26]:
st_pub_fw: Firewall = network.get_node_by_hostname(hostname="ST_PUB-FW")
st_dmz_pub_srv_web: Server = network.get_node_by_hostname(hostname="ST_DMZ-PUB-SRV-WEB")

In [27]:
# Some Tech Firewall --> Some Tech DMZ public web server

st_pub_fw.ping(st_dmz_pub_srv_web.network_interface[1].ip_address)

True

In [28]:
# Some Tech Firewall --> Public DNS Server

st_pub_fw.ping(isp_pub_srv_dns.network_interface[1].ip_address)

True

In [29]:
# Some Tech Firewall --> Internet Router (Port 4)

st_pub_fw.ping(isp_pub_rt_br.network_interface[4].ip_address)

True

## UC7 Network | SOME_TECH_INTRANET

In [30]:
st_intra_prv_rt_cr: Router = network.get_node_by_hostname(hostname="ST_INTRA-PRV-RT-CR")
st_intra_prv_rt_dr_1: Router = network.get_node_by_hostname(hostname="ST_INTRA-PRV-RT-DR-1")
st_intra_prv_rt_dr_2: Router = network.get_node_by_hostname(hostname="ST_INTRA-PRV-RT-DR-2")

In [31]:
# Some Tech Intranet Private Router CR --> Some Tech Public Firewall

st_intra_prv_rt_cr.ping(st_pub_fw.network_interface[3].ip_address)

True

In [32]:
# Some tech intranet router DR 1 --> Some Tech Intranet Router CR

st_intra_prv_rt_dr_1.ping(st_intra_prv_rt_cr.network_interface[1].ip_address)

True

In [33]:
# Some tech intranet router DR 2 --> Some Tech Intranet Router CR

st_intra_prv_rt_dr_2.ping(st_intra_prv_rt_cr.network_interface[1].ip_address)

True

In [34]:
# Some tech intranet router DR 1 --> Some tech intranet router DR 2

st_intra_prv_rt_dr_1.ping(st_intra_prv_rt_dr_2.network_interface[1].ip_address)

True

In [35]:
# Some tech intranet router DR 1 --> Some Tech Firewall (Internal Port)

st_intra_prv_rt_dr_1.ping(st_pub_fw.network_interface[3].ip_address)

True

In [36]:
# Some tech intranet router DR 1 --> Some Tech Firewall (DMZ Port)

st_intra_prv_rt_dr_1.ping(st_pub_fw.network_interface[2].ip_address)

True

In [37]:
# Some tech intranet router DR 1 --> Some Tech Firewall (External Port)

st_intra_prv_rt_dr_1.ping(st_pub_fw.network_interface[1].ip_address)

True

In [38]:
# Some Tech Intranet Router DR 1 --> Some Tech DMZ web-server

st_intra_prv_rt_dr_1.ping(target_ip_address=st_dmz_pub_srv_web.network_interface[1].ip_address)

True

In [39]:
# Some tech intranet router DR 1 --> Public Internet

st_intra_prv_rt_dr_1.ping(isp_pub_rt_br.network_interface[3].ip_address)

True

In [40]:
# Some tech intranet router DR 1 --> Public DNS

st_intra_prv_rt_dr_1.ping(isp_pub_srv_dns.network_interface[1].ip_address)

True

In [41]:
# Some Tech Intranet Router DR 1 --> Home Office PC 1

st_intra_prv_rt_dr_1.ping(target_ip_address=home_pub_pc_1.network_interface[1].ip_address)

True

In [42]:
# Some Tech Intranet Router DR 1 --> Remote Site PC 1

st_intra_prv_rt_dr_1.ping(target_ip_address=rem_pub_pc_1.network_interface[1].ip_address)

True

## UC7 Network | Some Tech Head Office

In [43]:
st_head_office_private_pc_1: Computer = network.get_node_by_hostname("ST_HO-PRV-PC-1")
st_head_office_private_pc_2: Computer = network.get_node_by_hostname("ST_HO-PRV-PC-2")
st_head_office_private_pc_3: Computer = network.get_node_by_hostname("ST_HO-PRV-PC-3")

In [44]:
# ST Home office PC 1 --> ST Home Office PC 2

st_head_office_private_pc_1.ping(st_head_office_private_pc_1.network_interface[1].ip_address)

True

In [45]:
# ST Home office PC 3 --> ST Router DR 2

st_head_office_private_pc_1.ping(st_intra_prv_rt_dr_2.network_interface[1].ip_address)

True

In [46]:
# ST Home office PC 1 --> Public DNS

st_head_office_private_pc_1.ping(isp_pub_srv_dns.network_interface[1].ip_address)

True

## UC7 Network | Some Tech Human Resources

In [47]:
st_human_resources_private_pc_1: Computer = network.get_node_by_hostname("ST_HR-PRV-PC-1")
st_human_resources_private_pc_2: Computer = network.get_node_by_hostname("ST_HR-PRV-PC-2")
st_human_resources_private_pc_3: Computer = network.get_node_by_hostname("ST_HR-PRV-PC-3")

In [48]:
# ST Human Resources PC 1 --> ST Human Resources PC 2

st_human_resources_private_pc_1.ping(st_human_resources_private_pc_2.network_interface[1].ip_address)

True

In [49]:
# ST Human Resources PC 2 --> # ST Human Resources PC 3

st_human_resources_private_pc_2.ping(st_human_resources_private_pc_3.network_interface[1].ip_address)

True

In [50]:
# ST Human Resources PC 1 --> ST Intranet Router DR 2

st_human_resources_private_pc_1.ping(st_intra_prv_rt_dr_2.network_interface[1].ip_address)

True

In [51]:
# ST Human Resources PC 1 --> Public DNS

st_human_resources_private_pc_1.ping(isp_pub_srv_dns.network_interface[1].ip_address)

True

## UC7 Network | Some Tech Data (Database / Database Backup)

In [52]:
st_data_private_server_database: Server = network.get_node_by_hostname("ST_DATA-PRV-SRV-DB")
st_data_private_server_storage: Server = network.get_node_by_hostname("ST_DATA-PRV-SRV-STORAGE")

In [53]:
# ST Data Private Database --> ST Data Private Storage

st_data_private_server_database.ping(st_data_private_server_storage.network_interface[1].ip_address)

True

In [54]:
# ST Data Private Storage --> Public DNS

st_data_private_server_storage.ping(isp_pub_srv_dns.network_interface[1].ip_address)

True

## UC7 Network | Some Tech Project A 

In [55]:
st_project_a_private_pc_1: Computer = network.get_node_by_hostname("ST_PROJ-A-PRV-PC-1")
st_project_a_private_pc_2: Computer = network.get_node_by_hostname("ST_PROJ-A-PRV-PC-2")
st_project_a_private_pc_3: Computer = network.get_node_by_hostname("ST_PROJ-A-PRV-PC-3")

In [56]:
# ST Private Project A PC 1 --> ST Private Project A PC 2

st_project_a_private_pc_1.ping(st_project_a_private_pc_2.network_interface[1].ip_address)

True

In [57]:
# ST Private Project A PC 2 --> ST Private Project A PC 3

st_project_a_private_pc_3.ping(st_project_a_private_pc_3.network_interface[1].ip_address)

True

In [58]:
# ST Private Project A PC 3 --> Public DNS

st_project_a_private_pc_3.ping(isp_pub_srv_dns.network_interface[1].ip_address)

True

In [59]:
# ST Private Project A PC 1 --> ST_INTRA-PRV-RT-CR

st_project_a_private_pc_1.ping(st_intra_prv_rt_cr.network_interface[1].ip_address)

True

In [60]:
# ST Private Project A PC 1 --> ST_INTRA-PRV-RT-DR-2

st_project_a_private_pc_1.ping("192.168.170.2")

True

## UC7 Network | Some Tech Project B

In [61]:
st_project_b_private_pc_1: Computer = network.get_node_by_hostname("ST_PROJ-B-PRV-PC-1")
st_project_b_private_pc_2: Computer = network.get_node_by_hostname("ST_PROJ-B-PRV-PC-2")
st_project_b_private_pc_3: Computer = network.get_node_by_hostname("ST_PROJ-B-PRV-PC-3")

In [62]:
# ST Private Project B PC 1 --> ST Private Project B PC 2

st_project_b_private_pc_1.ping(st_project_b_private_pc_2.network_interface[1].ip_address)

True

In [63]:
# ST Private Project B PC 2 --> ST Private Project B PC 3

st_project_b_private_pc_2.ping(st_project_b_private_pc_3.network_interface[1].ip_address)

True

In [64]:
# ST Private Project B PC 3 --> Public DNS

st_project_b_private_pc_3.ping(isp_pub_srv_dns.network_interface[1].ip_address)

True

## UC7 Network | Some Tech Project C

In [65]:
st_project_c_private_pc_1: Computer = network.get_node_by_hostname("ST_PROJ-C-PRV-PC-1")
st_project_c_private_pc_2: Computer = network.get_node_by_hostname("ST_PROJ-C-PRV-PC-2")
st_project_c_private_pc_3: Computer = network.get_node_by_hostname("ST_PROJ-C-PRV-PC-3")

In [66]:
# ST Private Project C PC 1 --> ST Private Project C PC 2

st_project_c_private_pc_1.ping(st_project_c_private_pc_2.network_interface[1].ip_address)

True

In [67]:
# ST Private Project C PC 2 --> ST Private Project C PC 3

st_project_c_private_pc_2.ping(st_project_c_private_pc_3.network_interface[1].ip_address)

True

In [68]:
# ST Private Project C PC 3 --> Public DNS

st_project_c_private_pc_3.ping(isp_pub_srv_dns.network_interface[1].ip_address)

True

In [69]:
# ST Private Project C PC 1 --> ST Private Project B PC 1

st_project_c_private_pc_1.ping(st_project_b_private_pc_1.network_interface[1].ip_address)

True

In [70]:
# ST Private Project B PC 1 --> ST Private Project A PC 1

st_project_b_private_pc_1.ping(st_project_a_private_pc_1.network_interface[1].ip_address)

True

In [71]:
# ST Private Project A PC 1 --> ST Head Office Private PC 1

st_project_a_private_pc_1.ping(st_head_office_private_pc_1.network_interface[1].ip_address)

True

In [72]:
# ST Private Project A PC 1 --> ST Human Resources Private PC 1

st_project_a_private_pc_1.ping(st_human_resources_private_pc_1.network_interface[1].ip_address)

True

## UC7 Network | Applications & Services

In [73]:
# dns-server

isp_pub_srv_dns.software_manager.show()

+---------------------------------------------------------------------------------------+
|                            ISP-PUB-SRV-DNS Software Manager                           |
+----------------------+-------------+-----------------+--------------+------+----------+
| Name                 | Type        | Operating State | Health State | Port | Protocol |
+----------------------+-------------+-----------------+--------------+------+----------+
| arp                  | Service     | RUNNING         | GOOD         | 219  | udp      |
| icmp                 | Service     | RUNNING         | GOOD         | None | icmp     |
| dns-client           | Service     | RUNNING         | GOOD         | 53   | tcp      |
| ntp-client           | Service     | RUNNING         | GOOD         | 123  | udp      |
| web-browser          | Application | RUNNING         | GOOD         | 80   | tcp      |
| nmap                 | Application | RUNNING         | GOOD         | None | none     |
| user-ses

In [74]:
# web-browser

st_project_a_web_browser = st_project_a_private_pc_1.software_manager.software["web-browser"]
st_project_a_web_browser.get_webpage()

True

In [75]:
# web-server

st_web_server = st_dmz_pub_srv_web.software_manager.software["web-server"]
st_web_server.sys_log.show()

+----------------------------------------------------------------------------------------------------------------------------------------------+
|                                                          ST_DMZ-PUB-SRV-WEB Sys Log                                                          |
+-------------------------+-------+------------------------------------------------------------------------------------------------------------+
| Timestamp               | Level | Message                                                                                                    |
+-------------------------+-------+------------------------------------------------------------------------------------------------------------+
| 2025-03-24 10:01:28,386 | INFO  | Sending echo reply to 192.168.100.2                                                                        |
| 2025-03-24 10:01:28,388 | INFO  | Received echo request from 192.168.160.2                                                      

In [76]:
# database-client

st_database_client = st_project_a_private_pc_1.software_manager.software["database-client"]
st_database_client.connect()

True

In [77]:
# database-service

st_database = st_data_private_server_database.software_manager.software["database-service"]
st_database.sys_log.show()

+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                   ST_DATA-PRV-SRV-DB Sys Log                                                                   |
+-------------------------+-------+------------------------------------------------------------------------------------------------------------------------------+
| Timestamp               | Level | Message                                                                                                                      |
+-------------------------+-------+------------------------------------------------------------------------------------------------------------------------------+
| 2025-03-24 10:01:28,630 | INFO  | Received ARP response for 192.168.220.2 from e8:50:df:b4:22:39 via Network Interface Port 1: 0f:bf:ca:f2:f9:80/192.168.220.3 |
| 2025-03-24 10:01:28,

In [78]:
# ftp-server

st_ftp_server = st_data_private_server_storage.software_manager.software["ftp-server"]

st_ftp_server.sys_log.show()

+----------------------------------------------------------------------------------------------------------------------------------------+
|                                                    ST_DATA-PRV-SRV-STORAGE Sys Log                                                     |
+-------------------------+-------+------------------------------------------------------------------------------------------------------+
| Timestamp               | Level | Message                                                                                              |
+-------------------------+-------+------------------------------------------------------------------------------------------------------+
| 2025-03-24 10:01:28,633 | INFO  | Sending echo reply to 192.168.220.2                                                                  |
| 2025-03-24 10:01:28,634 | INFO  | Received echo request from 192.168.220.3                                                             |
| 2025-03-24 10:01:28,635 |

In [79]:
# dns-client 

home_pub_pc_1: Computer = network.get_node_by_hostname("HOME-PUB-PC-1")
dns_client: DNSClient = home_pub_pc_1.software_manager.software["dns-client"]

dns_client.check_domain_exists(target_domain="some_tech.com")
dns_client.dns_cache.get("some_tech.com", None)
len(dns_client.dns_cache) == 1


True

## UC7 Network | Green & Red Agent (TAP001) default behaviour

In [80]:
for _ in range(100):
    env.step(0)

In [81]:
def print_agent_actions_except_do_nothing(agent_name):
    print(f"\n{agent_name} actions this episode:")
    for item in env.game.agents[agent_name].history:
        if item.action != "do-nothing":
            node, application = 'unknown', 'unknown'
            if (item.parameters.get('node_name')) is not None:
                node = item.parameters.get('node_name')
            if (item.parameters.get('application_name')) is not None:
                application = item.parameters.get('application_name')
            print(f"Step: {item.timestep}, action: {item.action}, node: {node}, application: {application}, response: {item.response.status}")

print_agent_actions_except_do_nothing("HOME_WORKER-1-DB")


HOME_WORKER-1-DB actions this episode:
Step: 4, action: node-application-execute, node: HOME-PUB-PC-1, application: database-client, response: success
Step: 8, action: node-application-execute, node: HOME-PUB-PC-1, application: database-client, response: success
Step: 13, action: node-application-execute, node: HOME-PUB-PC-1, application: database-client, response: success
Step: 18, action: node-application-execute, node: HOME-PUB-PC-1, application: database-client, response: success
Step: 21, action: node-application-execute, node: HOME-PUB-PC-1, application: database-client, response: success
Step: 25, action: node-application-execute, node: HOME-PUB-PC-1, application: database-client, response: success
Step: 30, action: node-application-execute, node: HOME-PUB-PC-1, application: database-client, response: success
Step: 35, action: node-application-execute, node: HOME-PUB-PC-1, application: database-client, response: success
Step: 39, action: node-application-execute, node: HOME-PUB

In [82]:
green_agent = env.game.agents.get("HOME_WORKER-1-DB")
print(f"Green Agent Ref: {green_agent.config.ref} Reward: {green_agent.reward_function.total_reward}")

Green Agent Ref: HOME_WORKER-1-DB Reward: 48.0


In [83]:
print_agent_actions_except_do_nothing("HOME_WORKER-1-WEB")


HOME_WORKER-1-WEB actions this episode:
Step: 2, action: node-application-execute, node: HOME-PUB-PC-1, application: web-browser, response: success
Step: 3, action: node-application-execute, node: HOME-PUB-PC-1, application: web-browser, response: success
Step: 6, action: node-application-execute, node: HOME-PUB-PC-1, application: web-browser, response: success
Step: 15, action: node-application-execute, node: HOME-PUB-PC-1, application: web-browser, response: success
Step: 17, action: node-application-execute, node: HOME-PUB-PC-1, application: web-browser, response: success
Step: 19, action: node-application-execute, node: HOME-PUB-PC-1, application: web-browser, response: success
Step: 26, action: node-application-execute, node: HOME-PUB-PC-1, application: web-browser, response: success
Step: 28, action: node-application-execute, node: HOME-PUB-PC-1, application: web-browser, response: success
Step: 48, action: node-application-execute, node: HOME-PUB-PC-1, application: web-browser,

In [84]:
green_agent = env.game.agents.get("HOME_WORKER-1-WEB")
print(f"Green Agent Ref: {green_agent.config.ref} Reward: {green_agent.reward_function.total_reward}")

Green Agent Ref: HOME_WORKER-1-WEB Reward: 19.599999999999962


In [85]:
red_agent = env.game.agents.get("attacker")
red_agent.logger.show()

+--------------------------------------------------------------------------------------------------------+
|                                         attacker Behaviour Log                                         |
+-----------+-------+------------------------------------------------------------------------------------+
| Time Step | Level | Message                                                                            |
+-----------+-------+------------------------------------------------------------------------------------+
| 61        | INFO  | TAP001 attacker has opted for a Port scan!                                         |
| 61        | INFO  | TAP001 attacker located the target but not yet confirmed a valid database service. |
| 66        | INFO  | Found a valid target on 192.168.220.3!                                             |
| 66        | INFO  | TAP001 attacker has opted for a Port scan!                                         |
| 66        | INFO  | TAP001 attacker

In [86]:
# successful TAP001 kill chain

st_project_a_private_pc_1.file_system.show(full=True)

st_data_private_server_database.file_system.show(full=True)

+---------------------------------------------------------------------------------------------+
|                                ST_PROJ-A-PRV-PC-1 File System                               |
+---------------------------------+---------+---------------+-----------------------+---------+
| File Path                       | Size    | Health status | Visible health status | Deleted |
+---------------------------------+---------+---------------+-----------------------+---------+
| downloads/malware_dropper.ps1   | 0 B     | GOOD          | NONE                  | False   |
| exfiltration_folder/database.db | 4.77 MB | GOOD          | NONE                  | False   |
| root                            | 0 B     | GOOD          | NONE                  | False   |
+---------------------------------+---------+---------------+-----------------------+---------+
+----------------------------------------------------------------------------------+
|                          ST_DATA-PRV-SRV-DB File 